In [1]:
!pip install autogluon
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import randint
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than

In [2]:
!pip uninstall lightgbm -y
!pip install lightgbm --no-cache-dir



Found existing installation: lightgbm 4.2.0
Uninstalling lightgbm-4.2.0:
  Successfully uninstalled lightgbm-4.2.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 62.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

# Load data
train_df = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')

# Display first few rows
train_df.head()
test_df.head()


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [4]:
# Extract horsepower and engine size from the 'engine' column
def extract_engine_features(engine_str):
    try:
        hp = float(engine_str.split('HP')[0].split()[-1])
        size = float(engine_str.split('L')[0].split()[-1])
    except ValueError:
        hp, size = None, None
    return pd.Series([hp, size])

train_df[['engine_hp', 'engine_size']] = train_df['engine'].apply(extract_engine_features)
test_df[['engine_hp', 'engine_size']] = test_df['engine'].apply(extract_engine_features)

# Drop the 'engine' column after feature extraction
train_df = train_df.drop(columns=['engine'])
test_df = test_df.drop(columns=['engine'])


In [5]:
# Handle missing values in train_df (including the 'price' column)
for col in train_df.select_dtypes(include=['float64', 'int64']).columns:
    train_df[col].fillna(train_df[col].mean(), inplace=True)

# Handle missing values in test_df (excluding the 'price' column, since it doesn't exist)
for col in test_df.select_dtypes(include=['float64', 'int64']).columns:
    test_df[col].fillna(test_df[col].mean(), inplace=True)

# Fill missing values in categorical columns with the most frequent value for both train_df and test_df
for col in train_df.select_dtypes(include=['object']).columns:
    train_df[col].fillna(train_df[col].mode()[0], inplace=True)

for col in test_df.select_dtypes(include=['object']).columns:
    test_df[col].fillna(test_df[col].mode()[0], inplace=True)


/tmp/ipykernel_23/3929515891.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(train_df[col].mean(), inplace=True)
/tmp/ipykernel_23/3929515891.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [6]:
# Convert categorical columns to category dtype
for col in train_df.select_dtypes(include=['object']).columns:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')


In [7]:
predictor = TabularPredictor(label='price', eval_metric='rmse').fit(
    train_df,
    presets='best_quality',
    time_limit=600,  # 10 minutes
    verbosity=3,  # More detailed output
    excluded_model_types=['KNN'],
    ag_args_fit={'num_gpus': 2}
)


No path specified. Models will be saved in: "AutogluonModels/ag-20240903_201044"
Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
GPU Count:          2
Memory Avail:       30.00 GB / 31.36 GB (95.7%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 2},
 'auto_stack': True,
 'excluded_model_types': ['KNN'],
 'num_bag_sets': 1,
 'verbosity': 3}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 2},
 'auto_stack': True,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging'

(_ray_fit pid=564) [50]	valid_set's rmse: 82256
(_ray_fit pid=564) [100]	valid_set's rmse: 82201


(_ray_fit pid=593) 	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=593) [LightGBM] [Fatal] bin size 1665 cannot run on GPU
(_ray_fit pid=593) Warning: GPU mode might not be installed for LightGBM, GPU training raised an exception. Falling back to CPU training...Refer to LightGBM GPU documentation: https://github.com/Microsoft/LightGBM/tree/master/python-package#build-gpu-versionOne possible method is:	pip uninstall lightgbm -y	pip install lightgbm --install-option=--gpu


(_ray_fit pid=593) [50]	valid_set's rmse: 83184.1
(_ray_fit pid=593) [100]	valid_set's rmse: 83064.9
(_ray_fit pid=593) [150]	valid_set's rmse: 82974
(_ray_fit pid=593) [200]	valid_set's rmse: 82950.7


(_ray_fit pid=622) 	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=622) [LightGBM] [Fatal] bin size 1670 cannot run on GPU
(_ray_fit pid=622) Warning: GPU mode might not be installed for LightGBM, GPU training raised an exception. Falling back to CPU training...Refer to LightGBM GPU documentation: https://github.com/Microsoft/LightGBM/tree/master/python-package#build-gpu-versionOne possible method is:	pip uninstall lightgbm -y	pip install lightgbm --install-option=--gpu


...

(_ray_fit pid=1376) 	Training S1F7 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=1376) [LightGBM] [Fatal] bin size 1665 cannot run on GPU
(_ray_fit pid=1376) Warning: GPU mode might not be installed for LightGBM, GPU training raised an exception. Falling back to CPU training...Refer to LightGBM GPU documentation: https://github.com/Microsoft/LightGBM/tree/master/python-package#build-gpu-versionOne possible method is:	pip uninstall lightgbm -y	pip install lightgbm --install-option=--gpu


(_ray_fit pid=1376) [50]	valid_set's rmse: 76387.2


(_ray_fit pid=1405) 	Training S1F8 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=1405) [LightGBM] [Fatal] bin size 1667 cannot run on GPU
(_ray_fit pid=1405) Warning: GPU mode might not be installed for LightGBM, GPU training raised an exception. Falling back to CPU training...Refer to LightGBM GPU documentation: https://github.com/Microsoft/LightGBM/tree/master/python-package#build-gpu-versionOne possible method is:	pip uninstall lightgbm -y	pip install lightgbm --install-option=--gpu


(_ray_fit pid=1405) [50]	valid_set's rmse: 82418.9


(_dystack pid=275) Saving AutogluonModels/ag-20240903_201044/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L2/utils/oof.pkl
(_dystack pid=275) Saving AutogluonModels/ag-20240903_201044/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L2/model.pkl
(_dystack pid=275) 	-73240.5392	 = Validation score   (-root_mean_squared_error)
(_dystack pid=275) 	50.91s	 = Training   runtime
(_dystack pid=275) 	0.77s	 = Validation runtime
(_dystack pid=275) 	7510.7	 = Inference  throughput (rows/s | 20948 batch size)
(_dystack pid=275) Saving AutogluonModels/ag-20240903_201044/ds_sub_fit/sub_fit_ho/models/trainer.pkl
(_dystack pid=275) Skipping LightGBM_BAG_L2 due to lack of time remaining.
(_dystack pid=275) Saving AutogluonModels/ag-20240903_201044/ds_sub_fit/sub_fit_ho/models/trainer.pkl
(_dystack pid=275) Skipping RandomForestMSE_BAG_L2 due to lack of time remaining.
(_dystack pid=275) Saving AutogluonModels/ag-20240903_201044/ds_sub_fit/sub_fit_ho/models/trainer.pkl
(_dystack pid=275) Skipping CatBoost_BA

In [8]:
# Print model performance
results = predictor.fit_summary()
print(results)

# Get leaderboard
lb = predictor.leaderboard()
print(lb)


Loading: AutogluonModels/ag-20240903_201044/models/LightGBMXT_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/LightGBM_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/RandomForestMSE_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/CatBoost_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/WeightedEnsemble_L2/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/LightGBMXT_BAG_L2/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/LightGBM_BAG_L2/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/WeightedEnsemble_L3/model.pkl


*** Summary of fit() ***
Estimated performance of each model:
                    model     score_val              eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     WeightedEnsemble_L3 -72768.336340  root_mean_squared_error      10.104682  374.546050                0.002832           0.210679            3       True          8
1       LightGBMXT_BAG_L2 -72819.671931  root_mean_squared_error       9.643301  321.681062                0.834290          55.293656            2       True          6
2     WeightedEnsemble_L2 -72821.825567  root_mean_squared_error       8.646410  228.687319                0.003193           0.159864            2       True          5
3       LightGBMXT_BAG_L1 -72879.577450  root_mean_squared_error       1.427207   56.714458                1.427207          56.714458            1       True          1
4         LightGBM_BAG_L1 -73158.421654  root_mean_squared_error       0.687765   51.438

In [9]:
# Generate predictions for test data
test_predictions = predictor.predict(test_df)

# Create submission DataFrame
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'price': test_predictions
})

# Save submission file
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


Loading: AutogluonModels/ag-20240903_201044/models/CatBoost_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/LightGBMXT_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/LightGBM_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/RandomForestMSE_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/LightGBMXT_BAG_L2/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/LightGBM_BAG_L2/model.pkl
Loading: AutogluonModels/ag-20240903_201044/models/WeightedEnsemble_L3/model.pkl


In [10]:
submission_df.head()

,id,price
0,188533,18465.087891
1,188534,82734.875000
2,188535,55964.460938
3,188536,35058.062500
4,188537,30713.298828
